# Parallel reduction

<a href="https://colab.research.google.com/github/mark-hobbs/articles/blob/main/cuda/parallel-reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See [this post](https://github.com/googlecolab/colabtools/issues/5081) to understand compatability issues with Google Colab and Numba CUDA

Literature:
- [Optimising parallel reduction in CUDA](https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf)

## Reduce bond forces to particle forces

Bond forces can be stored as a bondlist or neighbour list

- bondlist [n_bonds, 2]
- neighbourlist [n_particles, n_family_members]

Reduce:
- particles.forces [n_particles, 1]

In [1]:
import numpy as np
from numba import njit, prange

try:
    import google.colab
    !git clone https://github.com/mark-hobbs/articles.git
    import os
    os.chdir('articles/cuda')  # Navigate to the cuda subdirectory
except ImportError:
    pass  # Already local, no need to clone

import utils

In [2]:
np.random.seed(42)
n_particles = 1500000
n_family_members = 128

bond_forces = np.random.rand(n_particles, n_family_members)

### Numpy and Numba

In [3]:
@utils.profile(runs=10)
def reduce_bond_forces_a(bond_forces):
    n_particles = bond_forces.shape[0]
    f = np.zeros((n_particles))
    for i in range(n_particles):
        f[i] = np.sum(bond_forces[i, :])
    return f

In [4]:
@utils.profile(runs=10)
def reduce_bond_forces_b(bond_forces):
    return np.sum(bond_forces, axis=1)

In [5]:
@utils.profile(runs=10)
@njit(parallel=True, fastmath=True)
def reduce_bond_forces_c(bond_forces):
    n_particles = bond_forces.shape[0]
    f = np.zeros((n_particles))
    for i in prange(n_particles):
        f[i] = np.sum(bond_forces[i, :])
    return f

In [6]:
f_a = reduce_bond_forces_a(bond_forces)
f_b = reduce_bond_forces_b(bond_forces)
f_c = reduce_bond_forces_c(bond_forces)
assert np.allclose(f_a, f_b) and np.allclose(f_b, f_c), "Results are not equal"

Function 'reduce_bond_forces_a' executed 10 time(s)
Average execution time: 1.8187 seconds
Min: 1.8049s, Max: 1.8471s

Function 'reduce_bond_forces_b' executed 10 time(s)
Average execution time: 0.0435 seconds
Min: 0.0386s, Max: 0.0663s

Function 'reduce_bond_forces_c' executed 10 time(s)
Average execution time: 0.0429 seconds
Min: 0.0128s, Max: 0.3071s



### Numba CUDA

In [7]:
from numba import cuda, float32

In [8]:
THREADS_PER_BLOCK = 256

@cuda.jit
def reduce_bond_forces_kernel(bond_forces, particle_forces):
    """
    Reduce bond forces to particle forces
    """

    shared = cuda.shared.array(THREADS_PER_BLOCK, dtype=bond_forces.dtype)

    particle = cuda.blockIdx.x
    tid = cuda.threadIdx.x
    n_family_members = bond_forces.shape[1]

    val = 0.0
    if tid < n_family_members:
        val = bond_forces[particle, tid]

    shared[tid] = val
    cuda.syncthreads()

    stride = THREADS_PER_BLOCK // 2
    while stride > 0:
        if tid < stride and tid + stride < n_family_members:
            shared[tid] += shared[tid + stride]
        cuda.syncthreads()
        stride //= 2

    if tid == 0:
        particle_forces[particle] = shared[0]

@utils.profile(runs=10)
def reduce_bond_forces_gpu(bond_forces):
    n_particles, n_family_members = bond_forces.shape

    bond_forces = cuda.to_device(bond_forces.astype(np.float32))
    f = cuda.device_array(n_particles, dtype=np.float32)

    reduce_bond_forces_kernel[n_particles, THREADS_PER_BLOCK](bond_forces, f)
    return f.copy_to_host()

In [9]:
utils.get_cuda_device_info()

Error retrieving CUDA device information: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


In [10]:
!uv pip install -q --system numba-cuda==0.4.0

In [11]:
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [12]:
f_gpu = reduce_bond_forces_gpu(bond_forces)
assert np.allclose(f_a, f_gpu), "Results are not equal"

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [ ]:
import time

def benchmark_kernel(bond_forces, num_runs=100):
    n_particles, n_family_members = bond_forces.shape

    bond_forces = cuda.to_device(bond_forces.astype(np.float32))
    f = cuda.device_array(n_particles, dtype=np.float32)

    # Warm up the kernel
    for _ in range(5):
        reduce_bond_forces_kernel[n_particles, THREADS_PER_BLOCK](bond_forces, f)

    cuda.synchronize()

    start_time = time.perf_counter()

    for _ in range(num_runs):
        reduce_bond_forces_kernel[n_particles, THREADS_PER_BLOCK](bond_forces, f)

    cuda.synchronize()

    end_time = time.perf_counter()

    avg_time = (end_time - start_time) / num_runs
    return avg_time, f.copy_to_host()

In [ ]:
cuda_event_time, result = benchmark_kernel(bond_forces, num_runs=100)
print(f"Kernel executed in {cuda_event_time:.4f} seconds")